Imports

In [19]:
#imports topo phases
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import scipy.sparse.linalg as sla
from scipy.linalg import eig
from tqdm import tqdm
import pickle 
import scipy.integrate as integrate
from scipy.integrate import quad
from matplotlib.colors import ListedColormap
from scipy.linalg import expm



Matrices, functions (todo: define a perm N-site function)

In [1]:
#unitary 2-qubit Shat=U=1j*Ureal

def Shat(A,T,G): 
    
    t = np.tanh(T/4 + G/4)
    tt = np.tanh(T/4 - G/4)
    Shatf = np.array([[1-t*tt,0,0,t+tt],[0,1+t*tt,t-tt,0],[0,-t+tt,1+t*tt,0],[-t-tt,0,0,1-t*tt]],dtype=complex)
    Shatg = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,-1]],dtype=complex)
    f = -(1j/4)*A*(2*np.cosh(T/2)+2*np.cosh(G/2))*(1/np.sinh(T/2))*(1/np.cosh(T/2))
    Shat = f*Shatf+Shatg
    Nalpha = 1+2*A**(2)*(np.cosh(T)/np.sinh(T))*(1/np.sinh(T))+2*A**(2)*np.cosh(G)*(1/np.sinh(T))**(2)
    Shat = (-1/np.sqrt(Nalpha))*Shat
    return Shat

def bwassembler_z(qc,matrix,layers,n):    #CHECKED#
    
    m=0
    while m < layers:
        i=0;j=0;    
        #even layer
        while i < n/2:
            qc.unitary(matrix, [2*i,2*i+1])
            i+=1;
        qc.barrier(range(n)) 
        #odd layer
        while j < n/2:
            
            if j == (n/2 - 1):
                
                qc.unitary(perm4site(matrix),[0,1,2,3],label='ZZ')
                
            else:
                qc.unitary(matrix, [2*j+1,2*j+2])
            
            j+=1;
        qc.barrier(range(n))
        
        m+=1
    
    return(qc)

def perm4site(U):   #CHECKED#
    
    P = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,-1]])
    
    P12 = np.kron(P,np.eye(4))
    P23 = np.kron(np.eye(2),np.kron(P,np.eye(2)))
    P34 = np.kron(np.eye(4),P)
    S34 = np.kron(np.eye(4),U)
    
    return P12@P23@P34@S34@P34@P23@P12

def perm8site(U):   #CHECKED#
    
    P = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,-1]])
    
    P12 = np.kron(P,np.eye(2**6))
    P23 = np.kron(np.eye(2),np.kron(P,np.eye(2**5)))
    P34 = np.kron(np.eye(2**2),np.kron(P,np.eye(2**4)))
    P45 = np.kron(np.eye(2**3),np.kron(P,np.eye(2**3)))
    P56 = np.kron(np.eye(2**4),np.kron(P,np.eye(2**2)))
    P67 = np.kron(np.eye(2**5),np.kron(P,np.eye(2**1)))
    P78 = np.kron(np.eye(2**6),P)
    S78 = np.kron(np.eye(2**6),U)
    
    return P12@P23@P34@P45@P56@P67@P78@S78@P78@P67@P56@P45@P34@P23@P12 


def prodstaten(n):   #CHECKED#
    
    X = np.array([[0,1],[1,0]])
    X0 = np.zeros((n,2**n,2**n),dtype = complex)
    
    for i in range(0,n):   
        X0[i]=np.kron(np.kron(np.eye(2**i),X),np.eye(2**(n-i-1)))
    operator = np.eye(2**n)
    for i in range(0,n//2):
        operator = (X0[2*i]-1j*X0[2*i+1])@operator
    return (1/np.sqrt(2**(n//2)))*operator


def perm_nsite(U,n):   #CHECKED#
    if n < 2:
        raise ValueError("Number of sites (n) must be at least 2.")

    # Define the permutation matrix P
    P = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0],
                    [0, 1, 0, 0],
                    [0, 0, 0, -1]])

    perm_list=[]
    # Construct permutation matrices and tensor products
    for i in range(0, n - 1):
        Pi = np.kron(np.eye(2 ** i), np.kron(P, np.eye(2 ** (n - i - 2))))
        perm_list.append(Pi)
        
    # Append U tensor product
    S_last = np.kron(np.eye(2 ** (n - 2)), U)
    perm_list.append(S_last)

    result = np.eye(2**n)
    #print(result, result.shape)
    for perm_matrix in perm_list:
        result = result @ perm_matrix

    perm_list.reverse()

    for perm_matrix in perm_list[1:]:
        result = result @ perm_matrix    
        
    return result      
    
def prodstaten(n):   #CHECKED#
    
    X = np.array([[0,1],[1,0]])
    X0 = np.zeros((n,2**n,2**n),dtype = complex)
    
    for i in range(0,n):   
        X0[i]=np.kron(np.kron(np.eye(2**i),X),np.eye(2**(n-i-1)))
    operator = np.eye(2**n)
    for i in range(0,n//2):
        operator = (X0[2*i]-1j*X0[2*i+1])@operator
    return (1/np.sqrt(2**(n//2)))*operator

#supercharges on 4 sites
def Q(T,G):
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1,0],[0,-1]])
    Id = np.array([[1,0],[0,1]])
    Q = np.exp(-(G+T)/4)*np.kron(X,Id)+np.exp((T+G)/4)*np.kron(Z,X)
    return Q
def Qbar(T,G):
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1,0],[0,-1]])
    Id = np.array([[1,0],[0,1]])
    Qbar = np.exp((T-G)/4)*np.kron(Y,Id)+np.exp(-(T-G)/4)*np.kron(Z,Y)

    return Qbar

#check supercharges commutation on 4 sites
def x(t):
    return np.exp(t/4)
def y(g):
    return np.exp(g/4)
def Qplus4site(t,g):
    Ql12 = x(t)*y(g)*X(1,4)+(1/(x(t)*y(g)))*Z(1,4)@X(2,4)
    Ql34 = x(t)*y(g)*Z(1,4)@Z(2,4)@X(3,4)+(1/(x(t)*y(g)))*Z(1,4)@Z(2,4)@Z(3,4)@X(4,4)
    return Ql12+Ql34
def Qminus4site(t,g):
    Qr12 = (y(g)/x(g))*Y(1,4)+(x(t)/y(g))*Z(1,4)@Y(2,4)
    Qr34 = (y(g)/x(g))*Z(1,4)@Z(2,4)@Y(3,4)+(x(g)/y(g))*Z(1,4)@Z(2,4)@Z(3,4)@Y(4,4)
    return Qr12+Qr34
#Tensor Paulis on nth site given L sites
def X(n,L):
    X = np.array([[0,1],[1,0]])
    return (np.kron(np.eye(2**(n-1)),np.kron(X,np.eye(2**(L-n)))))
def Y(n,L):
    Y = np.array([[0,-1j],[1j,0]])
    return (np.kron(np.eye(2**(n-1)),np.kron(Y,np.eye(2**(L-n)))))    
def Z(n,L):
    Z = np.array([[1,0],[0,-1]])
    return (np.kron(np.eye(2**(n-1)),np.kron(Z,np.eye(2**(L-n)))))    
print('Commutations for Ur: ',np.allclose(Q(T,G)@Shat(A,T,G)-Shat(A,T,G)@Q(-T,-G),0*np.eye(4)),
np.allclose(Qbar(T,G)@Shat(A,T,G)-Shat(A,T,G)@Qbar(-T,-G),0*np.eye(4)))


NameError: name 'np' is not defined

Commutation with circuit

In [37]:
qc = QuantumCircuit(4)
Qtot=Qplus4site(1,1)+Qminus4site(1,1)
op=Operator(bwassembler_z(qc,Shat(1,1,1),1,4)).data
np.allclose(Qtot@op-op@Qtot,np.eye(16)*0)

True

IBM stuff

In [20]:
basis_gates=['ecr', 'id', 'rz', 'sx', 'x']

dispcolors = {'cx':('#426299', '#000000'),
            'cswap':('#705399', '#000000'),
              'cp':('#CFFF8B', '#000000')}


####general imports
from qiskit import quantum_info
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import matplotlib.pyplot as plt
import numpy as np
import pickle
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
basis_gates=['ecr', 'id', 'rz', 'sx', 'x']
#InteractiveShell.ast_node_interactivity = "all"

#imports Aer/Estimator/Sampler
from qiskit_aer import AerSimulator
from qiskit.circuit.library import RealAmplitudes
aer_sim=AerSimulator()
AerSimulator().available_methods()
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler, QiskitRuntimeService , EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager 
from qiskit.quantum_info import SparsePauliOp

#execution SamplerV2
backend = aer_sim
shots=1000
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(qc)
#print('b')
with Session(backend=aer_sim) as session:
    sampler = Sampler(session=session)
    result = sampler.run([isa_qc], shots=shots).result()
# Get the results
    #print('c')
    counts = result[0].data.meas.get_counts()



#####execution EstimatorV2
#psi = RealAmplitudes(num_qubits=2, reps=2)
hamiltonian = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
theta = [0, 1, 1, 2, 3, 5]
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_psi = pm.run(psi)
isa_observables = hamiltonian.apply_layout(isa_psi.layout)
estimator = Estimator(backend=backend)
# calculate [ <psi(theta1)|hamiltonian|psi(theta)> ]
job = estimator.run([(isa_psi, isa_observables, [theta])])
pub_result = job.result()[0]
print(f"Expectation values: {pub_result.data.evs}")
#####


####noisy backend
from qiskit.providers.fake_provider import GenericBackendV2
backend = GenericBackendV2(8)

####evolve and latex output
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace, purity
# Set the initial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2**n)
# Evolve the state by the quantum circuit
state = state.evolve(qc)
#draw using latex
state.draw('latex')


NameError: name 'qc' is not defined

BdG Real Space Coefficients 

In [ ]:
#definition of all coefficients 
def sech(x):
    return 1/np.cosh(x)
def csch(x):
    return 1/np.sinh(x)

def Na(a,g,eps1):#check
    return (sech(g/2)-tanh(g/2)*sech(g/2)**3)/(a) +eps1
def Nb(a,g,eps2):#check
    return (sech(g/2)+tanh(g/2)*sech(g/2)**3)/(a) +eps2
def Jab(a,g):#check
    return (3*tanh(g/2)**(2)*sech(g/2)**(2)+sech(g/2)**(4))/(2*a)
def Jba(a,g):#check
    return (sech(g/2)**(4))/(2*a)
def Jaa(a,g):#check
    return (tanh(g/2)*sech(g/2)**(3))/(2*a)
def Jabl(a,g,d):#check
    return -(tanh(g/2)**(2)*sech(g/2)**(2))/(2*a)+d
def Sab(g):#check
    return sech(g/2)/2
def Sba(g):#check
    return (sech(g/2)**(3))/(2)
def Saa(g):#check
    return (tanh(g/2)*sech(g/2)**(2))/(2)
def Sabl(g):#check
    return -(tanh(g/2)**(2)*sech(g/2))/(2)

def MajBasis(n,a,g,eps1,eps2,d):
    rotator=np.kron((1/np.sqrt(2))*np.array([[1,1j],[1,-1j]]),np.eye(n))
    return rotator@HBdG(n,a,g,eps1,eps2,d)@rotator.conj().T

Pickle Load/Save

In [ ]:
#loading data
with open('filename', 'rb') as file:
    loaded_data = pickle.load(file)

#saving data
with open('filename', 'wb') as file: 
    pickle.dump(tosave, file)